In [1]:
import pickle

from pyboost import controllers as pb
from pyspark import SparkContext

sc = SparkContext(master="local[2]")

In [2]:
X_train, X_test, y_train, y_test = pickle.load(open("../experiments/higgs/data/higgs-data-1p.pickle"))

In [4]:
X = sc.parallelize(X_train)
y = sc.parallelize([y + y - 1 for y in  y_train])

In [5]:
nodes = pb.run_adaboost_adtree(y, X, T=50, quiet=False)

=== Iteration 1 ===
Score (sorted by index): [3598.6692489376783, 3631.9070265880573, 3441.198790966997, 3560.1983625849161, 3573.541660593889, 3564.1225654258978, 3567.7242786509369, 3567.7515384023918, 3581.4999308480465, 3581.2813881503971, 3578.9601841875947, 3577.0473733389545, 3591.3051626237502, 3508.5951604595366, 3383.8053348254584, 3435.0167394061054, 3579.3380089760194, 3571.4270129760107, 3569.6019618579985, 3567.9127793856551, 3577.0561312442924, 3574.6731319101154, 3551.3774878559102, 3570.1042001600076, 3569.1050563298763, 3481.3021594188913, 3543.6993140415088, 3543.2134048092571]
Purity (farther from 1.0 is better): (0.94408204085235847, 1.0218019297540868)
Predicts (farther from 0.0 is better): (-0.028771104465153546, 0.010783833247877524)
Split node: 0 (left)
Split index and value: 14 -0.641552388668 

=== Iteration 2 ===
Score (sorted by index): [3550.1393113169302, 3570.0329588389832, 3573.5197414024265, 3553.7737997257755, 3573.9271718089085, 3574.6604253811283, 3

In [8]:
from pyboost.utils import safe_comp

def test_error(root, y, X, max_index=None):    
    error = 0
    margin = 0.0
    for tX, ty in zip(X, y):
        ty = ty + ty - 1
        _m = root.run(tX, max_index=max_index)
        predict = safe_comp(_m)
        if safe_comp(predict, ty):
            error += 1
        margin += _m * ty
    return 1.0 - float(error) / len(y), margin

In [9]:
root = nodes[0]
train_accuracy, train_margin = test_error(root, y_train, X_train)
test_accuracy, test_margin = test_error(root, y_test, X_test)
print "Train accuracy:", train_accuracy, '\t',
print "Train margin:", train_margin
print "Test accuracy:", test_accuracy, '\t',
print "Test margin:", test_margin

Train accuracy: 0.63825 	Train margin: 12698.2670708
Test accuracy: 0.6359 	Test margin: 1195.14092894


In [10]:
# Verify if train margin keep increasing
for max_index in range(50):
    accuracy, margin = test_error(root, y_train, X_train, max_index=max_index)
    print max_index, accuracy, margin

0 0.52977 354.913841744
1 0.52977 385.934192161
2 0.53852 1359.31773702
3 0.54298 1370.17062048
4 0.54298 1370.56942544
5 0.60797 6291.59143967
6 0.61421 7142.59604483
7 0.61561 8155.10896066
8 0.61561 8230.28369902
9 0.61561 8224.02090308
10 0.61561 8378.75501079
11 0.61561 8381.12490821
12 0.61784 8820.63980155
13 0.61784 8820.6609609
14 0.61784 8820.63802345
15 0.61784 8820.58138808
16 0.61784 8820.60311011
17 0.61982 9282.15703191
18 0.61982 9279.93068667
19 0.61982 9276.50435304
20 0.61982 9275.57952342
21 0.62029 9284.51889993
22 0.63661 10837.522764
23 0.63661 10826.827142
24 0.63661 10812.8108529
25 0.63661 10800.7617397
26 0.63661 10786.9244944
27 0.63828 11896.3007595
28 0.63819 12110.200786
29 0.63819 12107.1011601
30 0.63825 12387.2555656
31 0.63825 12433.3273644
32 0.6382 12478.2516095
33 0.63826 12519.4445931
34 0.63823 12551.8192908
35 0.63825 12568.5541592
36 0.63823 12580.0032181
37 0.63824 12592.1736398
38 0.63822 12595.2545577
39 0.63822 12598.3021839
40 0.63822 1260